> Open-source model

- Llama 3: https://huggingface.co/meta-llama/Llama-3.3-70B-Instruct -> 7b
- Gemma 3: https://huggingface.co/google/gemma-3-27b-it -> 7b
- Mistral: https://huggingface.co/mistralai/Mixtral-8x7B-Instruct-v0.1 -> 7b
- Falcon: https://huggingface.co/tiiuae/falcon-180B -> 7b
- Vicuna: https://huggingface.co/lmsys/vicuna-13b-v1.5 -> 7b
- Qwen: https://huggingface.co/Qwen/Qwen3-235B-A22B -> 7b

- https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF

In [139]:
import numpy as np
import pandas as pd

import ast
import random

from huggingface_hub import InferenceClient
from IPython.display import Markdown

In [140]:
raw_link = pd.read_csv('postprocess.csv')
raw_tthc = pd.read_csv('data/raw_tthc.csv', dtype={'maThuTuc': str})

In [141]:
cols = ['link', 'Mã thủ tục hành chính liên quan', 'Số quyết định', 'Tên thủ tục', 'Cấp thực hiện', 'Loại thủ tục', 'Lĩnh vực', 
        'Trình tự thực hiện', 'Cách thức thực hiện', 'Thành phần hồ sơ', 'Đối tượng thực hiện', 'Cơ quan thực hiện', 
        'Cơ quan có thẩm quyền', 'Địa chỉ tiếp nhận hồ sơ', 'Cơ quan được ủy quyền', 'Cơ quan phối hợp', 'Kết quả thực hiện', 
        'Căn cứ pháp lý', 'Yêu cầu, điều kiện thực hiện', 'Từ khóa', 'Mô tả']

In [142]:
raw_tthc.columns = cols

In [143]:
hf_api_key = pd.read_csv('../../hf_api_key.txt', header=None).values[0][0]

In [144]:
client = InferenceClient(api_key=hf_api_key)

In [145]:
model_name = []

model_name.append("mistralai/Mixtral-8x7B-Instruct-v0.1")
model_name.append("meta-llama/Llama-3.3-70B-Instruct")
model_name.append("google/gemma-3-27b-it")
model_name.append("tiiuae/falcon-180B")
model_name.append("lmsys/vicuna-13b-v1.5")
model_name.append("Qwen/Qwen3-235B-A22B")

temperature = 0.7
top_p = 0.9
max_tokens = 512

In [146]:
pattern = []

pattern.append("Bạn đang cố gắng xác định xem câu trả lời có hiểu sai ngữ cảnh và mục đích của câu hỏi hay không.")
pattern.append("Bạn đang cố gắng xác định xem câu trả lời có sự mâu thuẫn về mặt thực tế so với kiến thức liên quan hay không. Một số thông tin trong câu trả lời có thể đã được bịa ra.")
pattern.append("Bạn đang cố gắng xác định xem câu trả lời có quá chung chung hoặc quá chi tiết so với mức độ cụ thể phù hợp để trả lời câu hỏi hay không.")
pattern.append("Bạn đang cố gắng xác định xem câu trả lời có thể được suy ra đúng từ kiến thức liên quan hay không.")

In [147]:
evaluate_template = \
"""Bạn sẽ đóng vai trò là một người đánh giá câu trả lời (answer judge). Với một câu hỏi và câu trả lời, mục tiêu của bạn là xác định xem câu trả lời được cung cấp có chứa thông tin không đúng sự thật hoặc thông tin ảo giác (hallucinated information) hay không. {pattern}

#Câu hỏi#: {question}

#Câu trả lời#: {answer}

#Đánh giá của bạn#:

Bạn nên cố gắng hết sức để xác định xem câu trả lời có chứa thông tin không đúng sự thật hoặc thông tin ảo giác hay không. Câu trả lời bạn đưa ra PHẢI là "Có" hoặc "Không".
"""

In [148]:
model = model_name[4]

df = pd.DataFrame(columns=['link', 'cauTraLoiAoGiac', 'pattern', 'danhGia'])

for i in range(len(raw_link)):
    p = raw_link['pattern'][i]
    question = raw_link['cauHoi'][i]
    answer = raw_link['cauTraLoiAoGiac'][i]
    prompt = evaluate_template.format(pattern=pattern[p], question=question, answer=answer)

    try:
        completion = client.chat.completions.create(model=model, messages=[{"role": "user", "content": prompt}], 
                                                    temperature=temperature, top_p=top_p, max_tokens=max_tokens)
        output = str(completion.choices[0].message.content)
    except Exception as e:
        output = ""
        print(e)
        break

    results = [raw_link['link'][i], answer, p, output]
    df.loc[i, ['link', 'cauTraLoiAoGiac', 'pattern', 'danhGia']] = results    

    print(f'Done of {len(raw_link)}: {i+1}', end='\r')

In [153]:
model = model_name[1]

i = random.randint(0, 3716)

prompt = evaluate_template.format(pattern=pattern[raw_link['pattern'][i]],
                                  question=raw_link['cauHoi'][i],
                                  answer=raw_link['cauTraLoiAoGiac'][i])

completion = client.chat.completions.create(model=model, messages=[{"role": "user", "content": prompt}], 
                                            temperature=temperature, top_p=top_p, max_tokens=max_tokens)
Markdown(completion.choices[0].message.content)

HfHubHTTPError: 402 Client Error: Payment Required for url: https://router.huggingface.co/hf-inference/models/meta-llama/Llama-3.3-70B-Instruct/v1/chat/completions (Request ID: Root=1-6824cd43-4136ace519aeb15736ebcf2d;bb856b1e-630b-4c42-a362-8177ce939f97)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.

In [2]:
import lmstudio as lms

model = lms.llm("llama-3.2-1b-instruct")
result = model.respond("What is the meaning of life?")

print(result)


LMStudioWebsocketError: 
    LM Studio is not reachable at ws://localhost:1234/llm (due to httpx.ConnectError: All connection attempts failed).
    Is LM Studio running?